In [1]:
import os
import rioxarray as rxr

# Define main path
main_path = r"Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\QA_flags\S2A_MSIL1C_20230103T134701_N0509_R024_T21HVB_20230103T170831.SAFE"
files = os.listdir(main_path)

# Find water band
water_band = next((band for band in files if "_water" in band), None)
water_mask = rxr.open_rasterio(os.path.join(main_path, water_band)) if water_band else None

# Initialize masks dictionary
masks = {
    "water": None,
    "cloud": None,
    "shadow": None,
    "glint": None,
    "swir_negative": None,
    "blue_negative": None,
    "green_negative": None,
    "red_negative": None,
    "nir_negative": None,
}

# File mapping for easy handling
file_mapping = {
    "_cloud_shadow.tif": "cloud_shadow",
    "_glint_mask.tif": "glint",
    "_swir_negative.tif": "swir_negative",
    "_vnir_neg_0.tif": "blue_negative",
    "_vnir_neg_1.tif": "green_negative",
    "_vnir_neg_2.tif": "red_negative",
    "_vnir_neg_3.tif": "nir_negative",
}

masks["water"] = water_mask

# Process files
for file in files:
    for suffix, key in file_mapping.items():
        if file.endswith(suffix):
            file_path = os.path.join(main_path, file)
            data = rxr.open_rasterio(file_path)
            data = data.rio.reproject_match(water_mask)
            
            if key == "cloud_shadow":
                masks["cloud"] = (data == 2).astype(int)
                masks["shadow"] = (data == 3).astype(int)
            else:
                masks[key] = data

water mask loaded.
cloud mask loaded.
shadow mask loaded.
glint mask loaded.
swir_negative mask loaded.
blue_negative mask loaded.
green_negative mask loaded.
red_negative mask loaded.
nir_negative mask loaded.


In [3]:
def create_qa_bitmask(water, cloud, shadow, glint_neg, neg_blue, neg_green, neg_red, neg_nir, neg_swir):
    
    # Encode cloud and shadow together (00 = clear, 01 = shadow, 10 = cloud, 11 = cloud + shadow)
    cloud_shadow = (cloud.astype(int) << 1) | (shadow.astype(int))
    
    # Construct the QA band with the new flags
    qa_band = (
        (water.astype(int) << 0) |         # Bit 0: Water
        (cloud_shadow << 1) |             # Bits 1-2: Cloud/Shadow Combined
        (glint_neg.astype(int) << 3) |    # Bit 3: Glint Correction
        (neg_blue.astype(int) << 4) |     # Bit 4: Negative Blue
        (neg_green.astype(int) << 5) |    # Bit 5: Negative Green
        (neg_red.astype(int) << 6) |      # Bit 6: Negative Red
        (neg_nir.astype(int) << 7) |      # Bit 7: Negative NIR
        (neg_swir.astype(int) << 8)       # Bit 8: Negative SWIR
    )
    
    return qa_band

# Example usage:
qa_band = create_qa_bitmask(masks["water"], masks["cloud"], masks["shadow"], masks["glint"], masks["blue_negative"], masks["green_negative"], masks["red_negative"], masks["nir_negative"], masks["swir_negative"])

In [4]:
qa_band.rio.to_raster(r"Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\QA_flags\qa_band.tif")

In [7]:
import rasterio
import numpy as np

def extract_mask(input_tiff, output_tiff, bit_range, bit_value):
    
    """
    Extract a mask from a GeoTIFF based on a specific bit range and bit value.

    Parameters:
    - input_tiff: Path to the input GeoTIFF file.
    - output_tiff: Path to save the output GeoTIFF file with the mask.
    - bit_range: A tuple (start_bit, end_bit) representing the bit range to extract (inclusive).
    - bit_value: The binary value to match in the selected bit range (e.g., '111').
    """
    
    # Step 1: Open the GeoTIFF
    with rasterio.open(input_tiff) as src:
        # Step 2: Read the image data (first band)
        image_data = src.read(1)

        # Step 3: Calculate the bitmask for the selected bit range
        start_bit, end_bit = bit_range
        num_bits = end_bit - start_bit + 1

        # Create the mask to extract the bits
        bitmask = (1 << num_bits) - 1  # e.g., for 3 bits: 111 (binary), which is 7 in decimal (2^0, 2^1, 2^2)

        # Shift the relevant bits to the right
        extracted_bits = (image_data >> start_bit) & bitmask

        # Convert the bit_value string (e.g., '111') to an integer
        bit_value_int = int(bit_value, 2)

        # Step 4: Create a binary mask where the extracted bits match the given bit_value
        mask = np.where(extracted_bits == bit_value_int, 1, 0)

        # Step 5: Write the mask to a new GeoTIFF file
        profile = src.profile
        profile.update(dtype=rasterio.uint16, count=1)

        with rasterio.open(output_tiff, 'w', **profile) as dst:
            dst.write(mask.astype(rasterio.uint16), 1)

    print(f"Mask for bit range {bit_range} and value {bit_value} exported as {output_tiff}")

In [9]:
extract_mask(r"Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\QA_flags\qa_band.tif", r"Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\QA_flags\qa_band_value.tif", bit_range=(1,2), bit_value='10')

Mask for bit range (1, 2) and value 10 exported as Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\QA_flags\qa_band_value.tif
